In [ ]:
from optimize import *

In [ ]:
# Python 3.5, 3.6
reeval_minimum(0, 50, 1, "forest", range(100, 105))
reeval_minimum(1, 50, 4, "dummy", range(100, 105))
reeval_minimum(2, 50, 3, "forest", range(100, 105))
reeval_minimum(3, 100, 4, "gbrt", range(100, 105))

In [ ]:
# Python 2.7
reeval_minimum(4, 140, 3, "gp", range(100, 105))
reeval_minimum(5, 50, 2, "gp", range(100, 105))
reeval_minimum(6, 50, 3, "forest", range(100, 105))
reeval_minimum(7, 50, 1, "gp", range(100, 105))
reeval_minimum(8, 50, 1, "forest", range(100, 105))